<a href="https://colab.research.google.com/github/graylan0/llama-looper/blob/main/Copy_of_llama_2_7b_chat_hf_sharded_bf16_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *About*

Forked from: https://colab.research.google.com/drive/1FxlUb_H6Xirhkx4RszAgHeb2uDW7oKIH by https://www.youtube.com/@vrsen

Authored by: RonanKMcGovern of https://research.trelis.com and https://huggingface.co/Trelis

# Install

In [1]:
# https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers
!pip install -q -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 7.0 MB/s eta 0:00:00
     ━

# Import

In [3]:
import transformers
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

### Google Drive Mounting below is optional. If you don't use it, remove cache_dir from the model and tokeniser

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
cache_dir = "/content/drive/My Drive/huggingface_cache"
os.makedirs(cache_dir, exist_ok=True) # Ensure the directory exists


# Load Quantized Model

In [4]:
model_id = "Trelis/Llama-2-7b-chat-hf-sharded-bf16-5GB" # sharded model by RonanKMcGovern. Change the model here to load something else.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
print(dir(tokenizer))


['SPECIAL_TOKENS_ATTRIBUTES', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_add_bos_token', '_add_eos_token', '_add_tokens', '_additional_special_tokens', '_auto_class', '_batch_encode_plus', '_bos_token', '_build_conversation_input_ids', '_call_one', '_cls_token', '_convert_encoding', '_convert_id_to_token', '_convert_token_to_id_with_added_voc', '_create_repo', '_decode', '_decode_use_source_tokenizer', '_encode_plus', '_eos_token', '_eventual_warn_about_too_long_sequence', '_eventually_correct_t5_max_length', '_from_pretrained', '_get_files_timestamps', '_get_padding_truncation_strategies', '_in_target_context_manager', '_mask_token', '_pad', '_pa

In [6]:
print("Current EOS token:", tokenizer.eos_token)


Current EOS token: </s>


In [7]:
# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

sample_string = 'hello'

# Tokenize the stringified JSON object
tokens = tokenizer.encode(sample_string)

# Count the number of tokens
token_count = len(tokens)

# Fetch BOS and EOS tokens
BOS_token_id = tokenizer.bos_token_id
EOS_token_id = tokenizer.eos_token_id
BOS_token = tokenizer.decode([BOS_token_id])
EOS_token = tokenizer.decode([EOS_token_id])

# Check and print BOS and EOS tokens
print(f"Beginning of the sequence: {tokens[0]} (BOS token: {BOS_token})")
print(f"End of the sequence: {tokens[-1]} (EOS token: {EOS_token})")
print(f"EOS token: {EOS_token}")

print(f"The number of tokens in the stringified JSON object is: {token_count}")

Beginning of the sequence: 1 (BOS token: <s>)
End of the sequence: 22172 (EOS token: </s>)
EOS token: </s>
The number of tokens in the stringified JSON object is: 2


In [8]:
print(model.config)


LlamaConfig {
  "_name_or_path": "Trelis/Llama-2-7b-chat-hf-sharded-bf16-5GB",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false
  },
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.32.0.dev0",
  "use_cache": true,
  "vocab_siz

In [12]:
def generate(index):
    system_prompt = 'You are a helpful assistant'
    user_prompt = ' Zero Noise Extrapolation'

    B_INST, E_INST = "[INST]", "[/INST]"
    B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

    # Define the roles and their corresponding prompts
    SYSTEM_ROLE = "system"
    USER_ROLE = "user"
    ASSISTANT_ROLE = "assistant"

    # Define your prompt template with the roles
    dialog = [
        {"role": SYSTEM_ROLE, "content": system_prompt.strip()},
        {"role": USER_ROLE, "content": user_prompt.strip()},
    ]

    # Transform dialog into a format compatible with Llama2
    dialog_transformed = [
        {
            "role": dialog[1]["role"],
            "content": f"{B_SYS}{dialog[0]['content']}{E_SYS}{B_INST}{dialog[1]['content']}{E_INST}",
        }
    ]

    # Concatenate the 'content' of the messages, maintaining the role sequence
    prompt = "".join([entry['content'] for entry in dialog_transformed])

    print("Prompt:")
    print(prompt)

    encoding = tokenizer(prompt, return_tensors="pt").to("cuda:0")
    output = model.generate(input_ids=encoding.input_ids, attention_mask=encoding.attention_mask, max_new_tokens=200, do_sample=True, temperature=0.01, eos_token_id=tokenizer.eos_token_id, top_k = 0)

    print()

    # Subtract the length of input_ids from output to get only the model's response
    output_text = tokenizer.decode(output[0], skip_special_tokens=False)

    print("Generated Assistant Response:")
    print(output_text)

    print()

In [13]:
generate(1)

Prompt:
<<SYS>>
You are a helpful assistant
<</SYS>>

[INST]Zero Noise Extrapolation[/INST]

Generated Assistant Response:
<s> <<SYS>>
You are a helpful assistant
<</SYS>>

[INST]Zero Noise Extrapolation[/INST]  Zero Noise Extrapolation (ZNE) is a machine learning technique used to estimate the values of a target variable for unsampled regions in a dataset. It is particularly useful when dealing with noisy or missing data, as it can provide accurate predictions for the target variable even when there is limited or noisy data available.
The basic idea behind ZNE is to use the available data to learn a mapping between the input features and the target variable. This mapping is then used to extrapolate the values of the target variable for unsampled regions. The key advantage of ZNE is that it can handle complex relationships between the input features and the target variable, and can provide accurate predictions even when the data is noisy or incomplete.
There are several approaches to Z